In [ ]:
#default_exp tests

In [ ]:
#hide
import sys
sys.path.append("..")
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
import unittest
import tempfile
import os
import gym
from deap import base, creator, tools
import optuna

from dpct.individual import DHPCTIndividual
from dpct.evolver import DHPCTEvolver
from dpct.optimizer import DHPCTOptimizer

# Unit Tests

> Comprehensive tests for the DPCT library components.

## DHPCTIndividual Tests

In [ ]:
class TestDHPCTIndividual(env_name="CartPole-v1",            env_props={},            levels=[                {'units': 10},                {'units': 5}            ],            activation_funcs=['relu', 'tanh'],            weight_types=['glorot_uniform', 'glorot_uniform']        )        def test_initialization(self):        """Test proper initialization of DHPCTIndividual"""        self.assertEqual(self.individual.env_name, "TestEnv")        self.assertEqual(self.individual.gym_name, "CartPole-v1")        self.assertEqual(len(self.individual.levels), 2)        self.assertEqual(len(self.individual.activation_funcs), 2)        self.assertEqual(len(self.individual.weight_types), 2)        self.assertIsNone(self.individual.env)        self.assertIsNone(self.individual.model)        def test_compile(self):        """Test compilation of the individual's model"""        self.individual.compile()        self.assertIsNotNone(self.individual.env)        self.assertIsNotNone(self.individual.model)        self.assertIsInstance(self.individual.model, Model)        def test_config(self):        """Test config method returns correct configuration"""        config = self.individual.config()        self.assertEqual(config['env_name'], "TestEnv")        self.assertEqual(config['gym_name'], "CartPole-v1")        self.assertEqual(len(config['levels']), 2)        self.assertEqual(config['levels'][0]['units'], 10)        self.assertEqual(config['levels'][1]['units'], 5)        def test_save_load_config(self):        """Test saving and loading configuration"""        with tempfile.NamedTemporaryFile(suffix='.json', delete=False) as tmp_file:            config_path = tmp_file.name                    try:            self.individual.save_config(config_path)            self.assertTrue(os.path.exists(config_path))                        with open(config_path, 'r') as f:                config = json.load(f)                            loaded_individual = DHPCTIndividual.from_config(config)            self.assertEqual(loaded_individual.env_name, "TestEnv")            self.assertEqual(loaded_individual.gym_name, "CartPole-v1")            self.assertEqual(len(loaded_individual.levels), 2)        finally:            os.unlink(config_path)        def test_run(self):        """Test running the individual in environment"""        rewards = self.individual.run(episodes=1, render=False)        self.assertEqual(len(rewards), 1)        self.assertIsInstance(rewards[0], (int, float))        def test_mate(self):        """Test mating operation"""        other_individual = DHPCTIndividual(env_name="CartPole-v1",            env_props={},            levels=[                {'units': 20},                {'units': 10}            ],            activation_funcs=['sigmoid', 'linear'],            weight_types=['he_uniform', 'he_normal']        )                offspring = self.individual.mate(other_individual)        self.assertIsInstance(offspring, DHPCTIndividual)        self.assertIn(offspring.env_name, ["TestEnv", "OtherEnv"])        self.assertEqual(len(offspring.levels), 2)            def test_mutate(self):        """Test mutation operation"""        original_config = self.individual.config()                # Use high mutation rate to ensure changes        self.individual.mutate(mutation_rate=1.0)                mutated_config = self.individual.config()        self.assertNotEqual(original_config['levels'][0]['units'],                            mutated_config['levels'][0]['units'])        def test_online_learning(self):        """Test online learning functionality"""        # Compile model to prepare for learning        self.individual.compile()                # Get original weights        original_weights = [layer.get_weights() for layer in self.individual.model.layers                            if len(layer.get_weights()) > 0]                # Run with online learning        self.individual.run(episodes=2, online_learning=True, learning_rate=0.1)                # Get new weights        new_weights = [layer.get_weights() for layer in self.individual.model.layers                       if len(layer.get_weights()) > 0]                # Check if weights have changed (at least one layer should have different weights)        weights_changed = False        for i in range(len(original_weights)):            if not np.array_equal(original_weights[i][0], new_weights[i][0]):                weights_changed = True                break                        self.assertTrue(weights_changed)

## DHPCTEvolver Tests

In [ ]:
class TestDHPCTEvolver(unittest.TestCase):    """Test cases for DHPCTEvolver class"""        def setUp(self):        """Set up test fixtures"""        # Create a template individual        self.template = DHPCTIndividual(env_name="CartPole-v1",            env_props={},            levels=[                {'units': 10},                {'units': 5}            ],            activation_funcs=['relu', 'tanh'],            weight_types=['glorot_uniform', 'glorot_uniform']        )                # Create evolver with small population for testing        self.evolver = DHPCTEvolver(            population_size=3,  # Small for quick tests            gens=2,             # Only 2 generations for quick tests            cx_prob=0.7,            mut_prob=0.2,            elite_size=1,            env_template=self.template        )        def test_initialization(self):        """Test proper initialization of DHPCTEvolver"""        self.assertEqual(self.evolver.population_size, 3)        self.assertEqual(self.evolver.gens, 2)        self.assertEqual(self.evolver.cx_prob, 0.7)        self.assertEqual(self.evolver.mut_prob, 0.2)        self.assertEqual(self.evolver.elite_size, 1)        self.assertIsNotNone(self.evolver.env_template)        self.assertIsNone(self.evolver.toolbox)        self.assertIsNone(self.evolver.stats)        self.assertIsNone(self.evolver.logbook)        def test_setup_evolution(self):        """Test setup of evolution toolbox"""        self.evolver.setup_evolution(evaluation_episodes=1, mutation_rate=0.1)        self.assertIsNotNone(self.evolver.toolbox)        self.assertIsNotNone(self.evolver.stats)        self.assertIsNotNone(self.evolver.logbook)        def test_run_evolution(self):        """Test running evolution process"""        # Setup with minimal evaluation for speed        self.evolver.setup_evolution(evaluation_episodes=1, mutation_rate=0.1)                # Run evolution        pop, log = self.evolver.run_evolution(verbose=False)                # Check results        self.assertEqual(len(pop), 3)  # Population size        self.assertEqual(len(log), 3)  # Initial + 2 generations        self.assertIsNotNone(self.evolver.best_individual)        self.assertIsInstance(self.evolver.best_individual, DHPCTIndividual)        def test_save_results(self):        """Test saving evolution results"""        # Setup and run evolution        self.evolver.setup_evolution(evaluation_episodes=1, mutation_rate=0.1)        self.evolver.run_evolution(verbose=False)                # Create temporary files for output        with tempfile.NamedTemporaryFile(suffix='.json', delete=False) as pop_file, \             tempfile.NamedTemporaryFile(suffix='.json', delete=False) as log_file, \             tempfile.NamedTemporaryFile(suffix='.json', delete=False) as best_file:                         pop_path = pop_file.name            log_path = log_file.name            best_path = best_file.name                try:            # Save results            self.evolver.save_results(                population_file=pop_path,                logbook_file=log_path,                best_file=best_path            )                        # Check files exist and contain valid JSON            self.assertTrue(os.path.exists(pop_path))            self.assertTrue(os.path.exists(log_path))            self.assertTrue(os.path.exists(best_path))                        with open(pop_path, 'r') as f:                pop_data = json.load(f)                self.assertEqual(len(pop_data), 3)  # Population size                            with open(log_path, 'r') as f:                log_data = json.load(f)                self.assertIn('generations', log_data)                self.assertIn('avg', log_data)                            with open(best_path, 'r') as f:                best_data = json.load(f)                self.assertIn('env_name', best_data)                self.assertIn('levels', best_data)        finally:            # Clean up            for path in [pop_path, log_path, best_path]:                if os.path.exists(path):                    os.unlink(path)

## DHPCTOptimizer Tests

In [ ]:
class TestDHPCTOptimizer(unittest.TestCase):
    """Test cases for DHPCTOptimizer class"""
    
    def setUp(self):
        """Set up test fixtures"""
        self.optimizer = DHPCTOptimizer(
            env_name="TestEnv",
            env_name="CartPole-v1",
            env_props={},
            n_trials=2,  # Minimal for testing
            timeout=None,
            db_storage=None
        )
    
    def test_initialization(self):
        """Test proper initialization of DHPCTOptimizer"""
        self.assertEqual(self.optimizer.env_name, "TestEnv")
        self.assertEqual(self.optimizer.gym_name, "CartPole-v1")
        self.assertEqual(self.optimizer.n_trials, 2)
        self.assertIsNone(self.optimizer.timeout)
        self.assertIsNone(self.optimizer.db_storage)
        self.assertIsNone(self.optimizer.study)
        self.assertIsNone(self.optimizer.best_params)
    
    def test_objective_function(self):
        """Test the objective function definition"""
        # Create a mock trial for testing
        trial = optuna.trial.FixedTrial({
            'n_levels': 2,
            'level_0_units': 10,
            'level_0_activation': 'relu',
            'level_0_weight': 'glorot_uniform',
            'level_1_units': 5,
            'level_1_activation': 'tanh',
            'level_1_weight': 'glorot_uniform'
        })
        
        # Set evaluation episodes to 1 for speed
        self.optimizer.evaluation_episodes = 1
        
        # Test objective function
        fitness = self.optimizer.define_objective(trial)
        self.assertIsInstance(fitness, (int, float))
    
    def test_run_optimization(self):
        """Test running optimization process"""
        # Run optimization with minimal evaluation for speed
        study = self.optimizer.run_optimization(evaluation_episodes=1, verbose=False)
        
        # Check results
        self.assertIsNotNone(study)
        self.assertEqual(len(study.trials), 2)  # n_trials
        self.assertIsNotNone(self.optimizer.best_params)
        self.assertIn('n_levels', self.optimizer.best_params)
    
    def test_get_best_individual(self):
        """Test getting best individual from optimization"""
        # Run optimization first
        self.optimizer.run_optimization(evaluation_episodes=1, verbose=False)
        
        # Get best individual
        individual = self.optimizer.get_best_individual()
        
        # Check results
        self.assertIsInstance(individual, DHPCTIndividual)
        self.assertEqual(individual.env_name, "TestEnv")
        self.assertEqual(individual.gym_name, "CartPole-v1")
        
        # Number of levels should match the optimized parameter
        n_levels = self.optimizer.best_params['n_levels']
        self.assertEqual(len(individual.levels), n_levels)
    
    def test_save_results(self):
        """Test saving optimization results"""
        # Run optimization first
        self.optimizer.run_optimization(evaluation_episodes=1, verbose=False)
        
        # Create temporary files for output
        with tempfile.NamedTemporaryFile(suffix='.json', delete=False) as params_file, \
             tempfile.NamedTemporaryFile(suffix='.json', delete=False) as ind_file:
             
            params_path = params_file.name
            ind_path = ind_file.name
        
        try:
            # Save results
            self.optimizer.save_results(
                best_params_file=params_path,
                best_individual_file=ind_path
            )
            
            # Check files exist and contain valid JSON
            self.assertTrue(os.path.exists(params_path))
            self.assertTrue(os.path.exists(ind_path))
            
            with open(params_path, 'r') as f:
                params_data = json.load(f)
                self.assertIn('n_levels', params_data)
                
            with open(ind_path, 'r') as f:
                ind_data = json.load(f)
                self.assertIn('env_name', ind_data)
                self.assertIn('levels', ind_data)
        finally:
            # Clean up
            for path in [params_path, ind_path]:
                if os.path.exists(path):
                    os.unlink(path)

In [ ]:
# Run the tests
unittest.main(argv=['first-arg-is-ignored'], exit=False)

# DPCT Unit Tests

> Unit tests for the Deep Perceptual Control Theory library components.

In [ ]:
import sys
sys.path.append("..")
import json
import unittest
import numpy as np
import tensorflow as tf
import gym
import optuna

In [ ]:
# Import DPCT components
from dpct.individual import DHPCTIndividual
from dpct.evolver import DHPCTEvolver
from dpct.optimizer import DHPCTOptimizer

## Test DHPCTIndividual

In [ ]:
class TestDHPCTIndividual(env_name="CartPole-v1",            env_props={"render_mode": "rgb_array"},            levels=[4, 3, 2]        )        def test_initialization(self):        self.assertEqual(self.individual.env_name, "CartPole")        self.assertEqual(self.individual.gym_name, "CartPole-v1")        self.assertEqual(self.individual.env_props, {"render_mode": "rgb_array"})        self.assertEqual(self.individual.levels, [4, 3, 2])        self.assertEqual(self.individual.weight_types, {"all": "float"})        def test_compile(self):        self.individual.compile()        self.assertIsNotNone(self.individual.env)        self.assertIsNotNone(self.individual.model)            def test_config(self):        config = self.individual.config()        self.assertIn("env", config)        self.assertIn("hierarchy", config)        self.assertIn("weights", config)        self.assertEqual(config["env"]["name"], "CartPole")        self.assertEqual(config["hierarchy"]["levels"], [4, 3, 2])        def test_from_config(self):        # Create a config        config = self.individual.config()                # Create a new individual from config        new_individual = DHPCTIndividual.from_config(config)                # Check if properties match        self.assertEqual(new_individual.env_name, self.individual.env_name)        self.assertEqual(new_individual.gym_name, self.individual.gym_name)        self.assertEqual(new_individual.levels, self.individual.levels)        def test_save_config(self):        # Save config to a temporary file        result = self.individual.save_config("temp_config.json")        self.assertTrue(result)                # Load the config file and check contents        with open("temp_config.json", "r") as f:            loaded_config = json.load(f)                self.assertEqual(loaded_config["env"]["name"], "CartPole")        def test_run(self):        self.individual.compile()        reward = self.individual.run(steps=10, early_termination=True)        self.assertIsInstance(reward, (int, float))        def test_run_with_training(self):        self.individual.compile()        reward = self.individual.run(steps=10, train=True, early_termination=True)        self.assertIsInstance(reward, (int, float))        def test_mate(self):        # Create another individual        other_individual = DHPCTIndividual(env_name="CartPole-v1",            env_props={"render_mode": "rgb_array"},            levels=[4, 3, 3]        )                # Mate individuals        child1, child2 = self.individual.mate(other_individual)                # Check that children are DHPCTIndividuals        self.assertIsInstance(child1, DHPCTIndividual)        self.assertIsInstance(child2, DHPCTIndividual)        def test_mutate(self):        # Get initial levels        initial_levels = self.individual.levels.copy()                # Force a structure mutation        np.random.seed(42)  # For reproducibility        self.individual.mutate(struct_prob=1.0, weight_prob=0.0)                # Check that levels have changed        self.assertNotEqual(self.individual.levels, initial_levels)        def test_evaluate(self):        self.individual.compile()        fitness = self.individual.evaluate(nevals=2)        self.assertIsInstance(fitness, (int, float))

In [ ]:
# Run the tests
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestDHPCTIndividual))

## Test DHPCTEvolver

In [ ]:
class TestDHPCTEvolver(unittest.TestCase):        def setUp(self):        # Create a test individual        self.individual = DHPCTIndividual(env_name="CartPole-v1",            env_props={"render_mode": "rgb_array"},            levels=[4, 3, 2]        )                # Create a test evolver        self.evolver = DHPCTEvolver(            pop_size=5,  # Small population for testing            generations=2,  # Few generations for testing            run_best=False,            save_arch_best=False        )                # Define a simple fitness function        def fitness_function(individual):            # Return a random fitness for testing            return np.random.random()                self.fitness_function = fitness_function        def test_initialization(self):        self.assertEqual(self.evolver.pop_size, 5)        self.assertEqual(self.evolver.generations, 2)        self.assertFalse(self.evolver.run_best)        def test_setup_evolution(self):        self.evolver.setup_evolution(            template_individual=self.individual,            fitness_function=self.fitness_function        )        self.assertIsNotNone(self.evolver.toolbox)        self.assertIsNotNone(self.evolver.stats)        self.assertIsNotNone(self.evolver.hof)        def test_run_evolution(self):        self.evolver.setup_evolution(            template_individual=self.individual,            fitness_function=self.fitness_function,            minimize=True        )                population, logbook, hof = self.evolver.run_evolution(verbose=False)                self.assertEqual(len(population), 5)  # Check population size        self.assertEqual(len(logbook), 3)  # Initial + 2 generations        self.assertEqual(len(hof), 1)  # Hall of fame size        def test_save_results(self):        # Setup and run evolution        self.evolver.setup_evolution(            template_individual=self.individual,            fitness_function=self.fitness_function        )        self.evolver.run_evolution(verbose=False)                # Save results        self.evolver.save_results("test_evolution")                # Check that files were created (minimal check)        import os        self.assertTrue(os.path.exists("test_evolution_stats.json"))

In [ ]:
# Run the tests
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestDHPCTEvolver))

## Test DHPCTOptimizer

In [ ]:
class TestDHPCTOptimizer(unittest.TestCase):        def setUp(self):        # Create a test individual        self.individual = DHPCTIndividual(env_name="CartPole-v1",            env_props={"render_mode": "rgb_array"},            levels=[4, 3, 2]        )                # Define evolution parameters for optimization        self.evolution_params = {            "pop_size": {"fixed": False, "type": "int", "min": 3, "max": 5},            "generations": {"fixed": True, "value": 2},            "minimize": {"fixed": True, "value": True}        }                # Create a test optimizer        self.optimizer = DHPCTOptimizer(            evolution_params=self.evolution_params,            n_trials=2  # Few trials for testing        )                # Define a simple fitness function        def fitness_function(individual):            # Return a random fitness for testing            return np.random.random()                self.fitness_function = fitness_function        def test_initialization(self):        self.assertEqual(self.optimizer.evolution_params, self.evolution_params)        self.assertEqual(self.optimizer.n_trials, 2)        self.assertIsNone(self.optimizer.timeout)        def test_define_objective(self):        self.optimizer.define_objective(            template_individual=self.individual,            fitness_function=self.fitness_function,            evaluation_budget=2        )                self.assertIsNotNone(self.optimizer._objective_func)        self.assertEqual(self.optimizer._template_individual, self.individual)        self.assertEqual(self.optimizer._fitness_function, self.fitness_function)        self.assertEqual(self.optimizer._evaluation_budget, 2)        def test_run_optimization(self):        self.optimizer.define_objective(            template_individual=self.individual,            fitness_function=self.fitness_function,            evaluation_budget=2        )                study = self.optimizer.run_optimization(verbose=False)                self.assertIsNotNone(study)        self.assertEqual(len(study.trials), 2)  # Two trials        self.assertIsNotNone(study.best_trial)        def test_get_best_params(self):        self.optimizer.define_objective(            template_individual=self.individual,            fitness_function=self.fitness_function        )                self.optimizer.run_optimization(verbose=False)        best_params = self.optimizer.get_best_params()                self.assertIn("pop_size", best_params)        self.assertIn("generations", best_params)        self.assertEqual(best_params["generations"], 2)  # Fixed value

In [ ]:
# Run the tests
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestDHPCTOptimizer))